# аналитика тираэт

In [1]:
import pandas as pd
from tqdm import trange, tqdm
import numpy as np
from IPython.display import display
import requests
from bs4 import BeautifulSoup as bs
import re

In [2]:
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_columns', 1000)

In [24]:
df = pd.read_csv('csv/tiraet_5_6_2024.csv')

In [4]:
brend = pd.read_excel('csv/brend.xlsx')

In [5]:
def lower(x):
    return x.lower()

In [6]:
brend['names'] = brend['name'].apply(lower)

In [7]:
brend_up = brend.name.unique()
brend_low = brend.names.unique()

загружаем и создаем переменную с брендами

In [8]:
def brends(x):
    xx = x.replace('\xa0', ' ').replace(',', ' ')
    xx = xx.split(' ')    
    for i in range(len(xx)-1):
        if xx[i].lower() in brend_low:
            return xx[i].lower()
        elif (str(xx[i].lower())+' '+str(xx[i+1].lower())) in brend_low:
            return (str(xx[i].lower())+' '+str(xx[i+1].lower()))    
    else:    
        return 'другой'

In [9]:
def category(x):
    xx = x.split('/')
    xx = (xx[4].replace('Komplektujuschie','КОМПЬЮТЕРНАЯ_ТЕХНИКА').
          replace('2873','МОБИЛЬНЫЕ_ТЕЛЕФОНЫ').
          replace('Kompjuternaja-mebel','ДЛЯ_ДОМА').
          replace('bytovaya','БЫТОВАЯ_ТЕХНИКА').
          replace('ohrannye-sistemy','БЕЗОПАСНОСТЬ').
          replace('svet-i-jelektrika','СВЕТОТЕХНИКА').
          replace('orgtehnika','ОРГТЕХНИКА').
          replace('PHoto--i-video','ФОТО_АУДИО').
          replace('setevoe-oborudovanie','LAN_ОБОРУДОВАНИЕ').
          replace('jelektropitanie','ЭЛЕКТРОПИТАНИЕ').
          replace('uslugi','СПОРТ').
          replace('avtojelektronika','АВТОТОВАРЫ').
          replace('rashodnye-materialy','РАСХОДНИКИ'))
    return xx    

In [20]:
def subcategory(x):
    try:
        reqx = requests.get(x)
        soup = bs(reqx.text, 'html.parser')    
        data = []
        data.append(soup.findAll('span', class_='breadcrumbs__item-name font_xs')[3].text)
        data.append(soup.findAll('span', class_='breadcrumbs__item-name font_xs')[4].text)
    except:
        data.append(soup.findAll('span', class_='breadcrumbs__item-name font_xs')[3].text)
        data.append(soup.findAll('span', class_='breadcrumbs__item-name font_xs')[3].text)                    
    return data

In [11]:
def group(x):
    xx = x.split('/')
    if len(xx) == 7:
        return xx[-3].replace('-','_')
    elif len(xx) == 8:
        return xx[-3].replace('-','_')
    elif len(xx) == 9:
        return xx[-4].replace('-','_')
    elif len(xx) == 10:
        return xx[-5].replace('-','_')
    elif len(xx) == 11:
        return xx[-6].replace('-','_')  

In [12]:
df['action_sale'] = 0

In [13]:
df.loc[df['old_price'] > 0,'action_sale'] = 1

создаем столбец который показывает является ли товар акционным(1 - акционный, 0 - не акционный)

In [14]:
df['sale'] = (100 - ((df['price']*100)/df['old_price'])).round(2) 

создаем столбец показывающий скидку в процентах

In [15]:
df['category'] = df['urls'].apply(category)

Создаем столбец показывающий категорию товаров

In [27]:
df = df[df['id']==82081]

In [21]:
df = df.head(5)

# Заглушка

In [28]:
df['subcategory'] = df['urls'].apply(subcategory)

C:\Users\user\AppData\Local\Temp\ipykernel_9580\2892512867.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['subcategory'] = df['urls'].apply(subcategory)


создаем столбец показывающий подкатегорию товаров

In [30]:
reqx = requests.get('https://tiraet.com/catalog/bytovaya/melkaja-bytovaja-tehnika/mashinki-dlja-udalenija-katyshkov/')
soup = bs(reqx.text, 'html.parser')   

In [31]:
try:
    x = soup.findAll('span', class_='breadcrumbs__item-name font_xs')[4].text
except:
    x = soup.findAll('span', class_='breadcrumbs__item-name font_xs')[3].text
    

In [32]:
x

'Машинки для удаления катышков'

In [33]:
df

,title,urls,datе,counts,id,price,old_price,subcategory
3723,Смартфон Realme C31 Silver 4/64GB (RMX3501) (Без связи IDC),https://tiraet.com/catalog/2873/6183/smartfon_realme_c31_silver_4_64gb_rmx3501_6_5_1600_720_ips_2_sim_karty_nfc_kamery_50_2_2_5_mpiks/,2024-06-05,Мало,82081,1749.0,NaN,"[СМАРТФОНЫ, Смартфон Realme C31 Silver 4/64GB (RMX3501) (Без связи IDC)]"


In [ ]:
df['brend'] = df['title'].apply(brends)